# 🚗 Projeto Completo: Previsão de Preço de Carros Usados

Este notebook apresenta a análise, limpeza, modelagem e previsão usando dados de veículos usados.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import chi2_contingency


## 1️⃣ 📥 Carregamento dos Dados e Limpeza Inicial

### 1.1️⃣ Carregar os dados
Carregamos o arquivo CSV com os dados dos veículos para análise inicial.

In [ ]:
df = pd.read_csv('vehicles.csv')

### 1.2️⃣ Análise descritiva inicial (opcional)
Visualização das primeiras linhas e estatísticas para entender o formato e as variáveis do dataset.

In [ ]:
# print(df.head())
# print(df.describe())

### 1.3️⃣ 🔧 Tratamento de Outliers na Variável 'price'
Valores de preço acima de 500.000 foram considerados outliers. Substituímos esses valores pela média dos preços dentro do limite para evitar que valores extremos prejudiquem o modelo.

In [ ]:
limite_preco = 500000  # limite superior para preço
df_sem_outlier = df[df['price'] <= limite_preco]
media_sem_outlier = df_sem_outlier['price'].mean()

df['price'] = df['price'].astype(float)
df.loc[df['price'] > limite_preco, 'price'] = media_sem_outlier


### 1.4️⃣ 🔧 Tratamento de Outliers na Variável 'odometer'
Valores de odômetro acima de 200.000 km foram considerados incomuns e substituídos pela média dos veículos dentro desse limite para manter dados consistentes.

In [ ]:
limite_odometer = 200000
out_odometer = df[df['odometer'] <= limite_odometer]
media_filtrada = out_odometer['odometer'].mean()

df.loc[df['odometer'] > limite_odometer, 'odometer'] = media_filtrada


### 1.5️⃣ 🔄 Preenchimento de valores ausentes em colunas numéricas
Para as colunas numéricas importantes, como 'year' e 'odometer', substituímos valores ausentes pela média, garantindo que não haja dados faltantes que possam prejudicar a análise.

In [ ]:
for col in ['year', 'odometer']:
    df[col] = df[col].fillna(df[col].mean())


### 1.6️⃣ ❌ Remoção de linhas com valores ausentes em colunas críticas
Removemos linhas que possuem valores ausentes em latitude, longitude e descrição, pois são essenciais para a análise e modelagem.

In [ ]:
df.dropna(subset=['lat', 'long', 'description'], inplace=True)

### 1.7️⃣ 🟠 Preenchimento de NaNs em colunas categóricas com 'unknown'
Para colunas categóricas importantes, substituímos valores ausentes por 'unknown', mantendo a consistência sem remover registros.

In [ ]:
colunas_para_preencher = [
    'manufacturer', 'model', 'fuel', 'title_status',
    'transmission', 'type', 'paint_color', 'condition',
    'cylinders', 'drive'
]
df[colunas_para_preencher] = df[colunas_para_preencher].fillna('unknown')


### 1.8️⃣ 🧹 Remoção de colunas irrelevantes ou com muitos dados faltantes
Colunas que não agregam valor para a modelagem ou possuem muitos dados ausentes foram removidas para simplificar o dataset.

In [ ]:
df.drop(columns=['VIN', 'size', 'county', 'url', 'region_url', 'image_url', 'description', 'posting_date'], inplace=True, errors='ignore')


### 1.9️⃣ 🧼 Verificação e remoção de linhas duplicadas
Identificamos e removemos registros duplicados para garantir qualidade e evitar vieses no modelo.

In [ ]:
print(f"Duplicadas: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)


## 2️⃣ 🧩 Preparação das Variáveis Categóricas

### 2.1️⃣ 🔍 Identificação das colunas categóricas
Selecionamos as colunas categóricas para codificação.

In [ ]:
cols_object = df.select_dtypes(include='object').columns
print(f"Colunas categóricas: {cols_object}")


### 2.2️⃣ 🎯 Manter as 10 categorias mais comuns em 'model' e 'region' e agrupar as outras em 'other'
Agrupamos as categorias menos frequentes em uma categoria única para evitar alta cardinalidade que pode prejudicar o modelo.

In [ ]:
top_10_model = df['model'].value_counts().nlargest(10).index.tolist()
df['model'] = df['model'].apply(lambda x: x if x in top_10_model else 'other')

top_10_region = df['region'].value_counts().nlargest(10).index.tolist()
df['region'] = df['region'].apply(lambda x: x if x in top_10_region else 'other')


### 2.3️⃣ 🧪 Teste Qui-Quadrado para variáveis categóricas
Avalia se as variáveis categóricas possuem relação estatisticamente significativa com o preço.

In [ ]:
cols_to_cod = [
    'region', 'manufacturer', 'model', 'condition', 
    'cylinders', 'fuel', 'title_status', 'transmission', 
    'drive', 'type', 'paint_color', 'state'
]

for col in cols_to_cod:
    tabela = pd.crosstab(df[col], df['price'])
    qui2, p, _, _ = chi2_contingency(tabela)
    print(f"{col} P-valor: {p:.2f}")


📝 **Interpretação:**
✅ Todas as variáveis categóricas testadas apresentaram p-valor < 0.05, indicando que possuem relação significativa com o preço.

## 3️⃣ 📊 Teste de Correlação para Variáveis Numéricas

In [ ]:
numeric = df[['price', 'year', 'odometer', 'lat', 'long']]
correlation = numeric.corr(numeric_only=True)['price'].drop('price')
print("\nCorrelação com Price:")
print(correlation)


📝 **Interpretação:**  
⏳ Ano (year): +0.28 (correlação positiva moderada) — Carros mais novos tendem a ter preço maior.  
🚗 Quilometragem (odometer): -0.43 (correlação negativa moderada) — Quanto maior a quilometragem, menor o preço.  
📍 Latitude e Longitude apresentam correlação muito baixa, sem impacto relevante.


## 4️⃣ ⚙️ Codificação One-Hot e Preparação dos Dados para Modelagem

In [ ]:
df = pd.get_dummies(df, columns=cols_to_cod, drop_first=True)

previsores = ['year', 'odometer'] + [col for col in df.columns if (
    col.startswith('region_') or
    col.startswith('manufacturer_') or
    col.startswith('model_') or
    col.startswith('condition_') or
    col.startswith('cylinders_') or
    col.startswith('fuel_') or
    col.startswith('title_status_') or
    col.startswith('transmission_') or
    col.startswith('drive_') or
    col.startswith('type_') or
    col.startswith('paint_color_') or
    col.startswith('state_')
)]

X = df[previsores].values
y = df['price'].values


## 5️⃣ 🧪 Treinamento e Avaliação do Modelo

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

modelo = DecisionTreeRegressor(random_state=42)
modelo.fit(X_train, y_train)

y_pred = modelo.predict(X_test)

print(f"\nMSE: {mean_squared_error(y_test, y_pred):,.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):,.2f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):,.2f}")

r2 = r2_score(y_test, y_pred)
print(f"R² (coeficiente de determinação): {r2:.3f}")


📝 **Interpretação do R²:**  
📉 R² < 0.3: modelo fraco, baixa capacidade explicativa.  
⚠️ 0.3 ≤ R² < 0.6: modelo razoável, precisa melhorar.  
✅ 0.6 ≤ R² < 0.8: modelo bom, explica boa parte da variabilidade.  
🌟 R² ≥ 0.8: modelo excelente, alta capacidade explicativa.

**Nosso Resultado:**  
📊 R² = 0.7 — O modelo demonstra boa capacidade de previsão, capturando a maior parte das variações no preço dos veículos. Esse resultado indica uma base robusta, porém, é possível aprimorar o modelo para alcançar maior precisão.


## 6️⃣ 🚗 Previsão Simples com Variáveis Numéricas

In [ ]:
previsores_num = ['year', 'odometer']
X_num = df[previsores_num].values
y = df['price'].values

X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=0.3, random_state=42)

modelo_num = DecisionTreeRegressor(random_state=42)
modelo_num.fit(X_train, y_train)

novo_carro_num = [[2015, 60000]]  # Ano 2015 e 60.000 km rodados (previsores)

preco_previsto = modelo_num.predict(novo_carro_num)[0]
print(f"Preço previsto só com dados numéricos: ${preco_previsto:,.2f}")


📝 **Observação:**  
Este é um exemplo simples de previsão usando somente as variáveis numéricas mais relevantes, útil para testes rápidos e validações iniciais.


# 📝 Observações Finais

Este projeto apresenta uma limpeza e tratamento de dados detalhados e completos, incluindo:

- Identificação e tratamento de outliers nas variáveis principais (price e odometer) para evitar distorções.  
- Tratamento cuidadoso de valores ausentes com estratégias diferentes para numéricos e categóricos.  
- Agrupamento das categorias menos frequentes para evitar alta cardinalidade.  
- Análises estatísticas com teste Qui-Quadrado e correlação para validar relações com o target.  
- Aplicação de técnicas adequadas para codificação e preparação dos dados.  
- Treinamento e avaliação de modelo de árvore de decisão com métricas claras e interpretação profissional.  
- Inclusão de exemplo prático para previsão simples, facilitando entendimento e testes.

O R² foi adotado como principal métrica para avaliação da performance, dado que é uma métrica interpretável para problemas de regressão, indicando o percentual de variância explicada pelo modelo.
